# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# Import tools and libraries
from dotenv import load_dotenv
from tavily import TavilyClient
import os
import json
import re
from pydantic import BaseModel
from openai import OpenAI
import chromadb
from langchain.tools import tool

In [ ]:
# Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [ ]:


# ------------------------------
# Chroma setup
# ------------------------------
chroma_client = chromadb.PersistentClient(path="chroma_db_jupiter")

# ------------------------------
# RETRIEVE_GAME TOOL
# ------------------------------
@tool("retrieve_game", return_direct=False)
def retrieve_game(query: str, n_results: int = 3):
    """
    Semantic search: Finds the most relevant games from the vector DB.

    Args:
        query (str): A question about the game industry.

    Returns:
        List[Dict]: Each result contains:
            - Platform (e.g., Game Boy, PS5, Xbox 360)
            - Name (name of the game)
            - YearOfRelease
            - Description
    """
    try:
        collection = chroma_client.get_collection("games_collection_new")

        results = collection.query(
            query_texts=query,
            n_results=n_results,
            include=["documents", "metadatas"]
        )

        metadatas = results.get("metadatas", [[]])[0]
        documents = results.get("documents", [[]])[0]

        output = []
        for meta, doc in zip(metadatas, documents):
            output.append({
                "Platform": meta.get("Platform"),
                "Name": meta.get("Name"),
                "YearOfRelease": meta.get("YearOfRelease"),
                "Description": meta.get("Description"),
                "document": doc
            })

        return output

    except Exception as e:
        return [{"error": str(e)}]


#### Evaluate Retrieval Tool

In [ ]:

class EvaluationReport(BaseModel):
    useful: bool
    description: str

@tool("evaluate_retrieval", return_direct=False)
def evaluate_retrieval(question: str, retrieved_docs: list[dict]):
    """
    LLM judge to evaluate whether retrieved docs are sufficient.
    """

    prompt = f"""
You are an evaluation assistant. Your task is to decide whether the retrieved documents are sufficient to answer the user's question.

Question:
{question}

Retrieved Documents:
{json.dumps(retrieved_docs, indent=2)}

Return ONLY valid JSON in this format:

{{"useful": true, "description": "..."}}
"""

    client = OpenAI(api_key=OPENAI_API_KEY, base_url=os.getenv("OPENAI_BASE_URL"))

    resp = client.chat.completions.create(
        model=os.getenv("EVAL_MODEL", "gpt-4o-mini"),
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )

    raw = resp.choices[0].message.content

    # --- JSON extraction ---
    try:
        parsed = json.loads(raw)
    except:
        match = re.search(r"\{[\s\S]*\}", raw)
        if not match:
            return {
                "useful": False,
                "description": f"Failed to extract JSON. Raw: {raw}"
            }
        parsed = json.loads(match.group(0))

    try:
        report = EvaluationReport.model_validate(parsed)
    except Exception as e:
        return {
            "useful": False,
            "description": f"Invalid JSON schema: {str(e)}. Raw: {parsed}"
        }

    return report.model_dump()



#### Game Web Search Tool

In [18]:

@tool("game_web_search", return_direct=False)
def game_web_search(question: str, max_results: int = 3):
    """
    Uses Tavily client to search the web for a gaming-related question.

    Semantic search: Finds relevant web results.
    Args:
        question: A question about the game industry.
        :param question:
        :param max_results:
    """
    try:
        tavily = TavilyClient(api_key=TAVILY_API_KEY)
        resp = tavily.search(
            query=question,
            include_answer=True,
            max_results=max_results
        )
        return {
            "answer": resp.get("answer"),
            "results": resp.get("results")
        }
    except Exception as e:
        return {
            "answer": None,
            "results": [],
            "error": str(e)
        }


### Agent

In [19]:
class AgentStateMachine:
    def __init__(self, retrieve_tool, evaluate_tool, web_tool, llm_tool):
        self.retrieve_tool = retrieve_tool
        self.evaluate_tool = evaluate_tool
        self.web_tool = web_tool
        self.llm_tool = llm_tool
        self.state = "start"
        self.last_retrieved = None
        self.last_evaluation = None

    def step(self, question):

        # START → RETRIEVE
        if self.state == "start":
            print("🔎 Retrieving from local DB...")
            results = self.retrieve_tool.run(question)   # ← correct
            self.last_retrieved = results
            self.state = "retrieved"
            return {"state": self.state, "retrieved": results}

        # RETRIEVED → EVALUATE
        elif self.state == "retrieved":
            print("🧠 Evaluating retrieval quality...")
            evaluation = self.evaluate_tool.run({
                "question": question,
                "retrieved_docs": self.last_retrieved
            })                                           # ← correct
            self.last_evaluation = evaluation

            if evaluation["useful"]:
                self.state = "done"
                return {
                    "state": "done",
                    "source": "local",
                    "data": self.last_retrieved,
                    "explanation": evaluation["description"]
                }
            else:
                self.state = "need_web"
                return {"state": "need_web"}

        # NEED_WEB → FINAL ANSWER
        elif self.state == "need_web":
            print("🌐 Local insufficient → searching the web...")
            web = self.web_tool.run(question)             # ← correct

            prompt = f"""
Local DB was insufficient.

Question: {question}

Local:
{json.dumps(self.last_retrieved, indent=2)}

Web:
{json.dumps(web, indent=2)}

Provide a concise, accurate answer.
"""

            answer = self.llm_tool(prompt)                # ← plain call
            self.state = "done"

            return {
                "state": "done",
                "source": [r["url"] for r in web.get("results", [])],
                "data": answer,
                "explanation": self.last_evaluation["description"]
            }

    def run(self, question):
        self.state = "start"

        s1 = self.step(question)
        if self.state == "done": return s1

        s2 = self.step(question)
        if self.state == "done": return s2

        return self.step(question)



In [20]:
client = OpenAI(
    api_key=OPENAI_API_KEY,
    base_url=os.getenv("OPENAI_BASE_URL")
)
agent = AgentStateMachine(
    retrieve_tool=retrieve_game,
    evaluate_tool=evaluate_retrieval,
    web_tool=game_web_search,
    llm_tool=lambda prompt: client.chat.completions.create(
        model=os.getenv("ANSWER_MODEL", "gpt-4.1-mini"),
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=300
    ).choices[0].message.content
)

queries = [
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for Playstation 5"
]

for q in queries:
    print("\n========================================")
    print("QUESTION:", q)
    print("========================================\n")

    result = agent.run(q)

    print('\n=== RESULT ===')
    print('Source:', result['source'])
    print('Explanation:', result['explanation'])
    print('Data:', result['data'])
    print('\n')


QUESTION: When Pokémon Gold and Silver was released?

🔎 Retrieving from local DB...
🧠 Evaluating retrieval quality...

=== RESULT ===
Source: local
Explanation: The retrieved document provides the release year of Pokémon Gold and Silver as 1999, which answers the user's question.
Data: [{'Platform': 'Game Boy Color', 'Name': 'Pokémon Gold and Silver', 'YearOfRelease': 1999, 'Description': 'Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.', 'document': '[Game Boy Color] Pokémon Gold and Silver (1999) - Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.'}, {'Platform': 'Game Boy Advance', 'Name': 'Pokémon Ruby and Sapphire', 'YearOfRelease': 2002, 'Description': 'Third-generation Pokémon games set in the Hoenn region, featuring new Pokémon and double battles.', 'document': '[Game Boy Advance] Pokémon Ruby and Sapphire (2002) - Third-generation Pokémon games set in the Hoenn region, featuring new Pokémon and 

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes